<a href="https://colab.research.google.com/github/chau-hm/A1111-on-colab/blob/main/StableDiffusionColab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# @title *  Stable Diffusion WebUI(Automatic1111) on Colabe by chauhm v1.0 *
# @markdown Step 0 
# @markdown
# @markdown Run this code block for the FIRST TIME only.
# @markdown 
# @markdown This block will create default folder structure on gdrive and download some commonly use embeddings and VAE.
# @markdown 
# @markdown Upload your LoRA, textual inversion , hypernetworks and VAE to the newly created folders before running next step.
# @markdown 
from google.colab import drive
from IPython.display import clear_output
import ipywidgets as widgets
import os
def inf(msg, style, wdth): inf = widgets.Button(description=msg, disabled=True, button_style=style, layout=widgets.Layout(min_width=wdth));display(inf)

# Mount gDrive
from google.colab import drive
drive.mount('/content/drive')

# -------------------------
# Create default folder structure and empty config files
# -------------------------
First_time_use = False # @param {type:"boolean"}
data_file_url = '/content/drive/MyDrive/StableDiffusionData'

if First_time_use and not os.path.exists(data_file_url):
  !mkdir $data_file_url
  !mkdir $data_file_url/models
  !mkdir $data_file_url/VAE
  !mkdir $data_file_url/settings
  !mkdir $data_file_url/Lora
  !mkdir $data_file_url/hypernetworks
  !mkdir $data_file_url/embeddings
  !mkdir $data_file_url/ControlNetModels
  !mkdir $data_file_url/outputs
  !mkdir $data_file_url/outputs/img2img-images
  !mkdir $data_file_url/outputs/txt2img-images
  #create empty config files
  !touch $data_file_url/settings/config.json
  !touch $data_file_url/settings/cache.json
  !touch $data_file_url/settings/ui-config.json
  !touch $data_file_url/settings/styles.csv
  #clone commonly use textual inversion, lora and vae
  !git clone https://huggingface.co/embed/negative $data_file_url/embeddings/negative
  !git clone https://huggingface.co/embed/lora $data_file_url/Lora/positive
  !wget -P $data_file_url/VAE https://huggingface.co/stabilityai/sd-vae-ft-mse-original/resolve/main/vae-ft-mse-840000-ema-pruned.safetensors 

clear_output()
inf('\u2714 Done','success', '50px')

In [ ]:
# @title Prepare the environment
# @markdown Step 1A
# @markdown
# @markdown Install Automatic1111, extensions and controlnet, choose and download checkpoints, load LoRA and other necessary resources from gdrive, restore settings from gdrive
from google.colab import drive
from IPython.display import clear_output
import ipywidgets as widgets
import os
def inf(msg, style, wdth): inf = widgets.Button(description=msg, disabled=True, button_style=style, layout=widgets.Layout(min_width=wdth));display(inf)

# Check GPU Type
! nvidia-smi

# Mount gDrive
from google.colab import drive
if not os.path.exists("/content/drive"):
  drive.mount("/content/drive")

# -------------------------
# Install Stable Diffusion
# -------------------------
%cd /content
import os
from google.colab.output import eval_js
os.environ['colab_url'] = eval_js("google.colab.kernel.proxyPort(7860, {'cache': false})")

!apt -y update -qq
!wget http://launchpadlibrarian.net/367274644/libgoogle-perftools-dev_2.5-2.2ubuntu3_amd64.deb
!wget https://launchpad.net/ubuntu/+source/google-perftools/2.5-2.2ubuntu3/+build/14795286/+files/google-perftools_2.5-2.2ubuntu3_all.deb
!wget https://launchpad.net/ubuntu/+source/google-perftools/2.5-2.2ubuntu3/+build/14795286/+files/libtcmalloc-minimal4_2.5-2.2ubuntu3_amd64.deb
!wget https://launchpad.net/ubuntu/+source/google-perftools/2.5-2.2ubuntu3/+build/14795286/+files/libgoogle-perftools4_2.5-2.2ubuntu3_amd64.deb
!apt install -qq libunwind8-dev
!dpkg -i *.deb
%env LD_PRELOAD=libtcmalloc.so
!rm *.deb

!apt -y install -qq aria2 libcairo2-dev pkg-config python3-dev
#!pip install torch==1.13.1+cu116 torchvision==0.14.1+cu116 torchaudio==0.13.1 torchtext==0.14.1 torchdata==0.5.1 --extra-index-url https://download.pytorch.org/whl/cu116 -U
#!pip install -q xformers==0.0.16 triton==2.0.0 -U
!pip install -q torch==2.0.0+cu118 torchvision==0.15.1+cu118 torchaudio==2.0.1+cu118 torchtext==0.15.1 torchdata==0.6.0 --extra-index-url https://download.pytorch.org/whl/cu118 -U
!pip install -q xformers==0.0.19 triton==2.0.0 -U

# -------------------------
# clone stable diffusion webui 
# -------------------------
!git clone -b v2.1 https://github.com/camenduru/stable-diffusion-webui

# -------------------------
# install extensions, add/remove extensions in this block
# -------------------------
%cd /content/stable-diffusion-webui/extensions
#!git clone -b v1.9 https://github.com/camenduru/deforum-for-automatic1111-webui /content/stable-diffusion-webui/extensions/deforum-for-automatic1111-webui
!git clone https://github.com/camenduru/stable-diffusion-webui-images-browser 
#!git clone https://github.com/AlUlkesh/stable-diffusion-webui-images-browser
#!git clone -b v1.9 https://github.com/camenduru/stable-diffusion-webui-huggingface /content/stable-diffusion-webui/extensions/stable-diffusion-webui-huggingface
#!git clone -b v1.9 https://github.com/camenduru/sd-civitai-browser /content/stable-diffusion-webui/extensions/sd-civitai-browser
!git clone https://github.com/kohya-ss/sd-webui-additional-networks 
#!git clone -b v1.9 https://github.com/camenduru/sd-webui-additional-networks /content/stable-diffusion-webui/extensions/sd-webui-additional-networks
!git clone https://github.com/camenduru/sd-webui-tunnels 
#!git clone -b v1.9 https://github.com/camenduru/batchlinks-webui /content/stable-diffusion-webui/extensions/batchlinks-webui
#!git clone -b v1.9 https://github.com/camenduru/stable-diffusion-webui-catppuccin /content/stable-diffusion-webui/extensions/stable-diffusion-webui-catppuccin
#!git clone -b v1.9 https://github.com/camenduru/a1111-sd-webui-locon /content/stable-diffusion-webui/extensions/a1111-sd-webui-locon
!git clone https://github.com/butaixianran/Stable-Diffusion-Webui-Civitai-Helper 
!git clone https://github.com/Mikubill/sd-webui-controlnet 
!git clone https://github.com/alemelis/sd-webui-ar 
!git clone https://github.com/hnmr293/sd-webui-llul 
!git clone https://github.com/Coyote-A/ultimate-upscale-for-automatic1111
!git clone https://github.com/ilian6806/stable-diffusion-webui-state
!git clone https://github.com/nonnonstop/sd-webui-3d-open-pose-editor
!git clone https://github.com/ashen-sensored/stable-diffusion-webui-two-shot
!git clone https://github.com/opparco/stable-diffusion-webui-composable-lora
!git clone https://github.com/jexom/sd-webui-depth-lib
#!git clone https://github.com/hako-mikan/sd-webui-supermerger
!git clone https://github.com/pkuliyi2015/multidiffusion-upscaler-for-automatic1111
#!git clone https://github.com/hnmr293/posex
#!git clone https://github.com/ashen-sensored/sd_webui_SAG
#!git apply --ignore-whitespace extensions/sd_webui_SAG/automatic1111-CFGDenoiser-and-script_callbacks-mod-for-SAG.patch
!git clone https://github.com/corbin-hayden13/SD-Prompt-Enhancer

!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://raw.githubusercontent.com/ototadana/sd-face-editor/main/scripts/face_editor.py -d /content/stable-diffusion-webui/scripts -o face_editor.py
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/embed/upscale/resolve/main/4x-UltraSharp.pth -d /content/stable-diffusion-webui/models/ESRGAN -o 4x-UltraSharp.pth
!wget https://raw.githubusercontent.com/camenduru/stable-diffusion-webui-scripts/main/run_n_times.py -O /content/stable-diffusion-webui/scripts/run_n_times.py
# -------------------------
# extensions end
# -------------------------

clear_output()
%cd /content/stable-diffusion-webui
!git reset --hard
!git -C /content/stable-diffusion-webui/repositories/stable-diffusion-stability-ai reset --hard


# @markdown ---
# @markdown *** A. Choose models to load:
%cd /content/stable-diffusion-webui/models/Stable-diffusion/
checkpoint_file_url = "/content/drive/MyDrive/StableDiffusionData/models"  

#copy thumbnails captured by Stable-Diffusion-Webui-Civitai-Helper 
!cp -av -u $checkpoint_file_url/*.preview.* /content/stable-diffusion-webui/models/Stable-diffusion/
!cp -av -u $checkpoint_file_url/*.civitai.info /content/stable-diffusion-webui/models/Stable-diffusion/


# @markdown ---
# @markdown Choose models to use

perfectWorld = False # @param {type:"boolean"}
if perfectWorld:
  !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://civitai.com/api/download/models/65269

theAllysMixIII = False # @param {type:"boolean"}
if theAllysMixIII:
  !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://civitai.com/api/download/models/12763

chilloutmix = True # @param {type:"boolean"}
if chilloutmix:
  !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://civitai.com/api/download/models/11745 

abyssorangemix3AOM3 = False # @param {type:"boolean"}
if abyssorangemix3AOM3:
  !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://civitai.com/api/download/models/11745

neverendingDreamNED = False # @param {type:"boolean"}
if neverendingDreamNED:
  !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://civitai.com/api/download/models/11925

neverendingDreamNED_inpainting = False # @param {type:"boolean"}
if neverendingDreamNED_inpainting:
  !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://civitai.com/api/download/models/12732

rpg_V4 = False # @param {type:"boolean"}
if rpg_V4:
  !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://civitai.com/api/download/models/7133

sunshinemix_sunlightmixPruned = False # @param {type:"boolean"}
if sunshinemix_sunlightmixPruned:
  !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://civitai.com/api/download/models/13510

dreamshaper = False # @param {type:"boolean"}
if dreamshaper:
  !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://civitai.com/api/download/models/43888

Lyriel = True # @param {type:"boolean"}
if Lyriel:
  !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://civitai.com/api/download/models/50127 

GhostMix = False # @param {type:"boolean"}
if GhostMix:
  !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://civitai.com/api/download/models/47142 

majicMIX_realistic = True # @param {type:"boolean"}
if majicMIX_realistic:
  !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://civitai.com/api/download/models/48289

majicMIX_fantasy = False # @param {type:"boolean"}
if majicMIX_fantasy:
  !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://civitai.com/api/download/models/49055

BRA = True # @param {type:"boolean"}
if BRA:
  !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://civitai.com/api/download/models/63786

Realistic_Vision = False # @param {type:"boolean"}
if Realistic_Vision:
  !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://civitai.com/api/download/models/29460

AbyssOrangeMix3 = False # @param {type:"boolean"}
if AbyssOrangeMix3:
  !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://civitai.com/api/download/models/17233

Anything = True # @param {type:"boolean"}
if Anything:
  !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://civitai.com/api/download/models/30163
# -------------------------
# add new model here
# find the download model id and replace {x} below
# -------------------------

#model_variable = True # @param {type:"boolean"}
#if model_variable:
#  !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://civitai.com/api/download/models/{x}
# -------------------------


# @markdown ---
# @markdown *** B. Choose to load ControlNet Models:
# @markdown - code borrowed from sdwuctw

control_canny = True # @param {type:"boolean"}
control_depth = True # @param {type:"boolean"}
control_mlsd = False # @param {type:"boolean"}
control_normal = False # @param {type:"boolean"}
control_openpose = True # @param {type:"boolean"}
control_scribble = False # @param {type:"boolean"}
control_seg = False # @param {type:"boolean"}
control_softedge = False # @param {type:"boolean"}
control_lineart = False # @param {type:"boolean"}
control_lineart_anime = False # @param {type:"boolean"}
control_ip2p = False # @param {type:"boolean"}
control_shuffle = False # @param {type:"boolean"}
control_inpaint = False # @param {type:"boolean"}
control_tile = True # @param {type:"boolean"}


%cd /content/stable-diffusion-webui/extensions

if os.path.exists('sd-webui-controlnet'):
  %cd sd-webui-controlnet/models
  if control_canny == True:
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/lllyasviel/ControlNet-v1-1/raw/main/control_v11p_sd15_canny.pth
  
  if control_depth == True:
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/lllyasviel/ControlNet-v1-1/raw/main/control_v11f1p_sd15_depth.pth
  
  if control_mlsd == True:
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/lllyasviel/ControlNet-v1-1/raw/main/control_v11p_sd15_mlsd.pth
  
  if control_normal == True:
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/lllyasviel/ControlNet-v1-1/raw/main/control_v11p_sd15_normalbae.pth
  
  if control_openpose == True:
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/lllyasviel/ControlNet-v1-1/raw/main/control_v11p_sd15_openpose.pth
  
  if control_scribble == True:
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/lllyasviel/ControlNet-v1-1/raw/main/control_v11p_sd15_scribble.pth
  
  if control_seg == True:
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/lllyasviel/ControlNet-v1-1/raw/main/control_v11p_sd15_seg.pth

  if control_softedge == True:
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/lllyasviel/ControlNet-v1-1/raw/main/control_v11p_sd15_softedge.pth
  
  if control_lineart == True:
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/lllyasviel/ControlNet-v1-1/raw/main/control_v11p_sd15_lineart.pth
  
  if control_lineart_anime == True:
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/lllyasviel/ControlNet-v1-1/raw/main/control_v11p_sd15s2_lineart_anime.pth
  
  if control_ip2p == True:
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/lllyasviel/ControlNet-v1-1/raw/main/control_v11e_sd15_ip2p.pth
  
  if control_shuffle == True:
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/lllyasviel/ControlNet-v1-1/raw/main/control_v11f1e_sd15_tile.pth

  if control_inpaint == True:
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/lllyasviel/ControlNet-v1-1/raw/main/control_v11p_sd15_inpaint.pth
  
  if control_tile == True:
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/lllyasviel/ControlNet-v1-1/raw/main/control_v11f1e_sd15_tile.pth
    


# @markdown ---
# Load lora, vae, embeddings and hypernetworks from gdrive

lora_file_url = "/content/drive/MyDrive/StableDiffusionData/Lora"  
vae_file_url = "/content/drive/MyDrive/StableDiffusionData/VAE"  
embeddings_file_url = "/content/drive/MyDrive/StableDiffusionData/embeddings"  
hypernetworks_file_url = "/content/drive/MyDrive/StableDiffusionData/hypernetworks"  
outputs_file_url = "/content/drive/MyDrive/StableDiffusionData/outputs"  

#!cp -av -u $lora_file_url "/content/stable-diffusion-webui/models/"
#!cp -av -u $vae_file_url "/content/stable-diffusion-webui/models/"
#!cp -av -u $embeddings_file_url "/content/stable-diffusion-webui/"
#!cp -av -u $hypernetworks_file_url "/content/stable-diffusion-webui/"

# -------------------------
#remove existing folders
# -------------------------
!rm -r /content/stable-diffusion-webui/models/Lora
!rm -r /content/stable-diffusion-webui/models/VAE
!rm -r /content/stable-diffusion-webui/embeddings
!rm -r /content/stable-diffusion-webui/hypernetworks

# -------------------------
#create softlinks instead
# -------------------------
!ln -s $lora_file_url /content/stable-diffusion-webui/models/
!ln -s $vae_file_url /content/stable-diffusion-webui/models/
!ln -s $embeddings_file_url /content/stable-diffusion-webui/
!ln -s $hypernetworks_file_url /content/stable-diffusion-webui/
# output files load faster in colab disk and provide a better experience 
#!ln -s $outputs_file_url /content/stable-diffusion-webui/

# -------------------------
#link user settings and config files from gdrive
# -------------------------
!ln -s "/content/drive/MyDrive/StableDiffusionData/settings/config.json" "/content/stable-diffusion-webui/config.json"
!ln -s "/content/drive/MyDrive/StableDiffusionData/settings/ui-config.json" "/content/stable-diffusion-webui/ui-config.json"
!ln -s "/content/drive/MyDrive/StableDiffusionData/settings/cache.json" "/content/stable-diffusion-webui/cache.json"
!ln -s "/content/drive/MyDrive/StableDiffusionData/settings/styles.csv" "/content/stable-diffusion-webui/styles.csv"

# -------------------------
#prepare environment
# -------------------------
!sed -i -e '''/    prepare_environment()/a\    os.system\(f\"""sed -i -e ''\"s/dict()))/dict())).cuda()/g\"'' /content/stable-diffusion-webui/repositories/stable-diffusion-stability-ai/ldm/util.py""")''' /content/stable-diffusion-webui/launch.py
!sed -i -e 's/\"sd_model_checkpoint\"\,/\"sd_model_checkpoint\,sd_vae\,CLIP_stop_at_last_layers\"\,/g' /content/stable-diffusion-webui/modules/shared.py

# @markdown ---

#!mkdir /content/stable-diffusion-webui/extensions/deforum-for-automatic1111-webui/models
inf('\u2714 Done','success', '50px')


In [ ]:
# @title Copy checkpoints from gdrive. DO NOT RUN unless necessary. 
# @markdown Step 1B
# @markdown
# @markdown This is an alternative way to load checkpoints files when CivitAI is down.
# @markdown
# @markdown DO NOT run this block if NO error occurs in previous step.
# @markdown
# @markdown Copy from gdrive by file id
# @markdown
# @markdown For more about file id, ref to https://www.youtube.com/watch?v=Mq8-WdcnzVo

Copy_ckpt_from_drive = False # @param {type: "boolean"}
# @markdown ---
# @markdown Choose models to use
perfectWorld = False # @param {type:"boolean"}
perfectWorld_file_id = '' # @param {type:"string"}

theAllysMixIII = False # @param {type:"boolean"}
theAllysMixIII_file_id = '' # @param {type:"string"}

chilloutmix = False # @param {type:"boolean"}
chilloutmix_file_id = '' # @param {type:"string"}

abyssorangemix3AOM3 = False # @param {type:"boolean"}
abyssorangemix3AOM3_file_id = '' # @param {type:"string"}

neverendingDreamNED = False # @param {type:"boolean"}
neverendingDreamNED3_file_id = '' # @param {type:"string"}

neverendingDreamNED_inpainting = False # @param {type:"boolean"}
neverendingDreamNED_inpainting_file_id = '' # @param {type:"string"}

rpg_V4 = False # @param {type:"boolean"}
rpg_V4_file_id = '' # @param {type:"string"}

sunshinemix_sunlightmixPruned = False # @param {type:"boolean"}
sunshinemix_sunlightmixPruned_file_id = '' # @param {type:"string"}

dreamshaper = False # @param {type:"boolean"}
dreamshaper_file_id = '' # @param {type:"string"}

Lyriel = False # @param {type:"boolean"}
Lyriel_file_id = '' # @param {type:"string"}

GhostMix = False # @param {type:"boolean"}
GhostMix_file_id = '' # @param {type:"string"}

majicMIX_realistic = False # @param {type:"boolean"}
majicMIX_realistic_file_id = '' # @param {type:"string"}

if Copy_ckpt_from_drive: 
  if perfectWorld:
    !gdown $perfectWorld_file_id
  if theAllysMixIII:
    !gdown $theAllysMixIII_file_id
  if chilloutmix:
    !gdown $chilloutmix_file_id
  if abyssorangemix3AOM3:
    !gdown $abyssorangemix3AOM3_file_id
  if neverendingDreamNED:
    !gdown $neverendingDreamNED3_file_id
  if neverendingDreamNED_inpainting:
    !gdown $neverendingDreamNED_inpainting_file_id
  if rpg_V4:
    !gdown $rpg_V4_file_id
  if sunshinemix_sunlightmixPruned:
    !gdown $sunshinemix_sunlightmixPruned_file_id
  if dreamshaper:
    !gdown $dreamshaper_file_id
  if Lyriel:
    !gdown $Lyriel_file_id
  if GhostMix:
    !gdown $GhostMix_file_id
  if majicMIX_realistic:
    !gdown $majicMIX_file_id


In [ ]:
# @title # Start Stable-Diffusion
# @markdown Step 2
# @markdown
# @markdown Start WebUI and backup necessary files when finishes
# @markdown
%cd /content/stable-diffusion-webui
!python launch.py --listen --xformers --enable-insecure-extension-access --theme dark --gradio-queue --multiple

clear_output()

# -------------------------
# backup output files
# -------------------------
!cp -av -u -f /content/stable-diffusion-webui/outputs/ /content/drive/MyDrive/StableDiffusionData/

# -------------------------
# backup config files
# -------------------------
from datetime import datetime
now = datetime.now()
timestamp = now.strftime("%Y%m%d%H%M%S")
!mkdir /content/drive/MyDrive/StableDiffusionData/settings/$timestamp
!cp -av -u /content/drive/MyDrive/StableDiffusionData/settings/config.json /content/drive/MyDrive/StableDiffusionData/settings/$timestamp/config.json
!cp -av -u /content/drive/MyDrive/StableDiffusionData/settings/ui-config.json /content/drive/MyDrive/StableDiffusionData/settings/$timestamp/ui-config.json
!cp -av -u /content/drive/MyDrive/StableDiffusionData/settings/cache.json /content/drive/MyDrive/StableDiffusionData/settings/$timestamp/cache.json
!cp -av -u /content/drive/MyDrive/StableDiffusionData/settings/styles.csv /content/drive/MyDrive/StableDiffusionData/settings/$timestamp/styles.csv

# -------------------------
#copy any new thumbnails back to gdrive
# -------------------------
!cp -av -u /content/stable-diffusion-webui/models/Stable-diffusion/*.preview.* $checkpoint_file_url
!cp -av -u /content/stable-diffusion-webui/models/Stable-diffusion/*.info $checkpoint_file_url

inf('\u2714 Remember to terminate the connection.','success', '250px')